# LiteLLM Proxy Connection Diagnosis

This notebook diagnoses connection issues to the LiteLLM Proxy.

It performs multiple steps to isolate why the API Key might be missing:
1. **Direct HTTP Request**: Verifies network, API Key, and Endpoint validity using `requests`.
2. **LiteLLM SDK Test (Standard)**: Standard `litellm.completion` call.
3. **LiteLLM SDK Test (Manually Constructed URL)**: Investigating exact URL endpoint.

In [1]:
import os
import requests
from litellm import completion
from dotenv import load_dotenv

# 1. Load Environment Variables
env_path = os.path.join(os.path.dirname(os.getcwd()), 'backend', '.env')
print(f"Loading .env from: {env_path}")
load_dotenv(env_path)

API_KEY = "sk-A_B3GLxZjx6IpAzUsFv7Zw"
API_BASE = "http://sacahan-ubunto:4000"
MODEL = "gpt-4o"

print(f"PROXY URL: {API_BASE}")
print(f"MODEL: {MODEL}")
print(f"API KEY: {API_KEY[:4]}...{API_KEY[-4:] if API_KEY else 'None'}")

Loading .env from: /Users/sacahan/Documents/workspace/TraitQuest/backend/.env
PROXY URL: http://sacahan-ubunto:4000
MODEL: gpt-4o
API KEY: sk-A...v7Zw


## Step 1: Direct connectivity test
This tests if the server is reachable and valid at the `/chat/completions` endpoint.
**Note:** We found the proxy listens at `/chat/completions`, NOT `/v1/chat/completions`.

In [ ]:
# Ensure base doesn't have trailing slash
if API_BASE.endswith('/'):
    base = API_BASE[:-1]
else:
    base = API_BASE

# Construct explicit URL
url = f"{base}/chat/completions"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

data = {
    "model": MODEL,
    "messages": [{"role": "user", "content": "Hello, direct test!"}]
}

print(f"Sending Direct Request to: {url}")
try:
    resp = requests.post(url, json=data, headers=headers, timeout=10)
    print(f"Status Code: {resp.status_code}")
    if resp.status_code == 200:
        print("✅ Direct Connection Successful")
        print("Response:", resp.json()['choices'][0]['message']['content'])
    else:
        print("❌ Direct Connection Failed")
        print("Body:", resp.text)
except Exception as e:
    print(f"❌ Exception during request: {e}")

## Step 2: OpenAI SDK Test


In [ ]:
# Enable verbose logging
from openai import OpenAI

client = OpenAI(
    base_url=API_BASE,  # 基礎 URL，不含 /chat/completions
    api_key=API_KEY,
)

try:
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"content": "Hello from LiteLLM!", "role": "user"}],
    )
    print("\n✅ OpenAI Success!")
    print(response.choices[0].message.content)
except Exception as e:
    print("\n❌ OpenAI Failed:")
    print(e)

## Step 3: LiteLLM SDK Test (Corrected URL)
If the previous test failed with 'Request blocked', it implies incorrect endpoint construction (e.g., trying to hit root).
We will try to force the path to `/chat/completions` by appending it to `api_base` IF we think LiteLLM strips it, OR verify path mechanics.

In [3]:
# Enable verbose logging
import litellm
litellm._turn_on_debug()

print("\n--- Experiment 5: Manually append /chat/completions to api_base ---")
# Normally 'openai' provider appends /chat/completions. But if API_BASE is treated as the full URL, we might need to specify exact path.
# However, if we append it, does LiteLLM append it again? Let's check logs.
manual_url = f"{API_BASE}/chat/completions"
print(f"Testing api_base='{manual_url}'")
try:
    response = litellm.completion(
        model=MODEL,
        messages=[{"content": "Hello from LiteLLM (Manual URL)!", "role": "user"}],
        base_url=API_BASE,
        api_key=API_KEY,
        # custom_llm_provider="openai",
        # extra_headers={
        #     "Content-Type": "application/json",
        #     "Authorization": f"Bearer {API_KEY}",
        # },
    )
    print("✅ Experiment 5 Success!")
    print(response.choices[0].message.content)
except Exception as e:
    print("❌ Experiment 5 Failed:")
    print(e)

15:43:38 - LiteLLM:DEBUG: utils.py:475 - 

15:43:38 - LiteLLM:DEBUG: utils.py:475 - Request to litellm:
15:43:38 - LiteLLM:DEBUG: utils.py:475 - litellm.completion(model='gpt-4o', messages=[{'content': 'Hello from LiteLLM (Manual URL)!', 'role': 'user'}], base_url='http://sacahan-ubunto:4000', api_key='sk-A_B3GLxZjx6IpAzUsFv7Zw')
15:43:38 - LiteLLM:DEBUG: utils.py:475 - 

15:43:38 - LiteLLM:DEBUG: utils.py:999 - Removing thought signatures from tool call IDs for non-Gemini model
15:43:38 - LiteLLM:DEBUG: litellm_logging.py:523 - self.optional_params: {}
15:43:38 - LiteLLM:DEBUG: utils.py:475 - SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
15:43:38 - LiteLLM:INFO: utils.py:3872 - 
LiteLLM completion() model= gpt-4o; provider = openai
15:43:38 - LiteLLM:DEBUG: utils.py:3875 - 
LiteLLM: Params passed to completion() {'model': 'gpt-4o', 'functions': None, 'function_call': None, 'temperature': None, 'top_p': None, 'n': None, 'stream': None, 'stream


--- Experiment 5: Manually append /chat/completions to api_base ---
Testing api_base='http://sacahan-ubunto:4000/chat/completions'


15:43:40 - LiteLLM:DEBUG: litellm_logging.py:1145 - RAW RESPONSE:
{"id": "chatcmpl-D3dSFVe2CcFBleD2sSKOOffAwwhn9", "choices": [{"finish_reason": "stop", "index": 0, "logprobs": null, "message": {"content": "Hello! \ud83d\udc4b It\u2019s great to connect with you via LiteLLM (Manual URL). How can I assist you today? \ud83d\ude0a", "refusal": null, "role": "assistant", "annotations": null, "audio": null, "function_call": null, "tool_calls": null, "provider_specific_fields": {"padding": "abcdefghijklmn"}}, "provider_specific_fields": {"content_filter_results": {"hate": {"filtered": false, "severity": "safe"}, "self_harm": {"filtered": false, "severity": "safe"}, "sexual": {"filtered": false, "severity": "safe"}, "violence": {"filtered": false, "severity": "safe"}}}}], "created": 1769759020, "model": "github_copilot/gpt-4o-2024-11-20", "object": "chat.completion", "service_tier": null, "system_fingerprint": "fp_3eed281ddb", "usage": {"completion_tokens": 28, "prompt_tokens": 16, "total_tok

✅ Experiment 5 Success!
Hello! 👋 It’s great to connect with you via LiteLLM (Manual URL). How can I assist you today? 😊


15:43:40 - LiteLLM:DEBUG: litellm_logging.py:1440 - response_cost: 0.00032
15:43:40 - LiteLLM:DEBUG: utils.py:5428 - checking potential_model_names in litellm.model_cost: {'split_model': 'github_copilot/gpt-4o-2024-11-20', 'combined_model_name': 'openai/github_copilot/gpt-4o-2024-11-20', 'stripped_model_name': 'github_copilot/gpt-4o-2024-11-20', 'combined_stripped_model_name': 'openai/github_copilot/gpt-4o-2024-11-20', 'custom_llm_provider': 'openai'}
15:43:40 - LiteLLM:DEBUG: utils.py:5698 - Error getting model info: This model isn't mapped yet. Add it here - https://github.com/BerriAI/litellm/blob/main/model_prices_and_context_window.json
15:43:40 - LiteLLM:DEBUG: litellm_logging.py:4574 - Model=github_copilot/gpt-4o-2024-11-20 is not mapped in model cost map. Defaulting to None model_cost_information for standard_logging_payload
15:43:40 - LiteLLM:DEBUG: litellm_logging.py:1874 - Logging Details LiteLLM-Success Call streaming complete
15:43:40 - LiteLLM:DEBUG: utils.py:5428 - checki